In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
!pip install -q torch torchvision
import torch
import torch.nn as nn
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from torch.optim import lr_scheduler

from tqdm.notebook import tqdm
from torchvision import models, transforms
import torchvision.datasets as datasets

In [ ]:
!pip install opendatasets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/arkadiyhacks/drinking-waste-classification")
od.download("https://www.kaggle.com/datasets/mostafaabla/garbage-classification")

100%|██████████| 1.50G/1.50G [00:11<00:00, 138MB/s]


100%|██████████| 239M/239M [00:02<00:00, 106MB/s] 


------shape of data:
drinking-waste-classification: <br>
  /drinking-waste-classification/Images_of_Waste/rawimgs/<br>
  folders = classes: AluCan, Glass, HDPEM, PET <br>
  filenames: classnum.jpg (exp: PET1,082.jpg)


garbage-classification: <br>
  /garbage-classification/garbage_classification/
  folders = classes: battery, biological, brown-glass, cardboard, clothes, green-glass, metal, paper, plastic, shoes, trash, white-glass <br>
  filenames: classnum.jpg (exp: green-glass627.jpg)


  The classes we want: <br>

AluCan, Glass, HDPEM, PET, Cardboard, Biological, C






In [ ]:
import os

'''
What datatypes are we looking at? What is the file structure?
'''

rawdir = '/content/drinking-waste-classification/Images_of_Waste/rawimgs/'

classes = sorted(os.listdir(rawdir))
print(classes)

n_files = {}
filetypes = {}
for dirname, _, filenames in os.walk(rawdir):
    n_files[dirname] = 0
    for filename in filenames:
        n_files[dirname] += 1   
        extension = filename.split('.')[-1]
        if extension in filetypes.keys():
            filetypes[extension] += 1
        else:    
            filetypes[extension] = 1

for directory, counts in n_files.items():
    print(f'number of files {directory} {counts}') 

for filetype, counts in filetypes.items():
    print(f'number of files of filetype {filetype} {counts}')     

['AluCan', 'Glass', 'HDPEM', 'PET']
number of files /content/drinking-waste-classification/Images_of_Waste/rawimgs/ 0
number of files /content/drinking-waste-classification/Images_of_Waste/rawimgs/PET 1508
number of files /content/drinking-waste-classification/Images_of_Waste/rawimgs/Glass 1232
number of files /content/drinking-waste-classification/Images_of_Waste/rawimgs/HDPEM 1028
number of files /content/drinking-waste-classification/Images_of_Waste/rawimgs/AluCan 1060
number of files of filetype jpg 3679
number of files of filetype JPG 1141
number of files of filetype HEIC 8


In [ ]:
import csv
ds1 = './drinking-waste-classification/Images_of_Waste/rawimgs/'
ds2 = './garbage-classification/garbage_classification/'

csvf = open('imagecats.csv', 'w')
fields = ['filename', 'category']
writer = csv.writer(csvf, delimiter=',')
writer.writerow(['filename', 'category'])

for dir in os.listdir(ds1):
  images = os.listdir(ds1+dir)
  classname = dir.lower()
  for im in images:
    if 'jpg' in im or 'JPG' in im:
      writer.writerow([ds1+dir+'/'+im, classname])

for dir in os.listdir(ds2):
  images = os.listdir(ds2+dir)
  classname = dir.lower()
  if 'glass' in dir:
    classname = 'glass'
  for im in images:
    if 'jpg' in im or 'JPG' in im:
      writer.writerow([ds2+dir+'/'+im, classname])


In [ ]:
import pandas as pd
imagecsv = pd.read_csv('imagecats.csv')

In [ ]:
from PIL import Image
from torch.utils.data import Dataset

class GarbageDataset(Dataset):
    def __init__(self, annot_file, class_mapping, transform=None, target_transform=None):
        self.transform = transform
        self.target_transform = target_transform
        self.annot_file = annot_file
        self.class_mapping = class_mapping

    def __len__(self):
        return self.annot_file.shape[0]

    def __getitem__(self, idx):
        image = Image.open(self.annot_file.iloc[idx, 0]).convert('RGB')
        label = self.class_mapping[self.annot_file.iloc[idx, 1]]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        if image.shape[0] != 3:
          print('WTF!!')
        return image, label

In [ ]:
img_transform = transforms.Compose([
              transforms.Resize(256),
              transforms.RandomResizedCrop(224),
              transforms.RandomHorizontalFlip(),
              transforms.ToTensor(),
              transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
          ])

from sklearn.model_selection import train_test_split
traincsv, testcsv = train_test_split(imagecsv, train_size=0.7)

class_map = {item:idx for idx, item in enumerate(np.unique(imagecsv.category))}

#train_data = GarbageDataset(imagecsv, class_map, transform=img_transform)
train_data = GarbageDataset(traincsv, class_map, transform=img_transform)
test_data = GarbageDataset(testcsv, class_map, transform=img_transform)

In [ ]:
class_map

{'alucan': 0,
 'battery': 1,
 'biological': 2,
 'cardboard': 3,
 'clothes': 4,
 'glass': 5,
 'hdpem': 6,
 'metal': 7,
 'paper': 8,
 'pet': 9,
 'plastic': 10,
 'shoes': 11,
 'trash': 12}

In [ ]:
batch = 64
train_dl = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch, shuffle=True)
test_dl = torch.utils.data.DataLoader(dataset=test_data, batch_size=batch, shuffle=True)

In [ ]:
def train_loop(dataloader, model, optim, loss_fn, num_epochs, device='cuda'):
  for epoch in range(num_epochs):
    epoch_loss = 0
    for x, y in tqdm(dataloader, total=len(dataloader), leave=False):
        x = x.to(device)
        y = y.to(device)
        optim.zero_grad()
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        epoch_loss += loss.item()
        loss.backward()
        optim.step()
    
    print(f'Epoch {epoch} loss: {epoch_loss}')


In [ ]:
def test_model(dataloader, model):
  acc = 0
  for x, y in dataloader:
    x = x.to('cuda')
    y = y.to('cuda')
    y_pred = torch.max(model(x), dim=1)[1]
    acc += (y == y_pred).sum()/y.shape[0]
  return acc/len(dataloader)

In [ ]:
def display_img(img):
  plt.imshow(torch.moveaxis(img, 0, 2))

In [ ]:
num_classes = 6
learning_rate = 0.002
num_epochs = 20
loss = nn.CrossEntropyLoss()

In [ ]:
model_tl = models.resnet18(pretrained=True)
num_feat_in = model_tl.fc.in_features
num_classes = len(class_map)
model_tl.fc = nn.Linear(num_feat_in, num_classes)

model_tl = model_tl.to('cuda')

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
optimizer = torch.optim.Adam(model_tl.parameters(), lr=learning_rate)
train_loop(train_dl, model_tl, optimizer, loss, num_epochs, 'cuda')

  0%|          | 0/222 [00:00<?, ?it/s]

Epoch 0 loss: 359.4081264734268


  0%|          | 0/222 [00:00<?, ?it/s]

Epoch 1 loss: 282.6631455421448


  0%|          | 0/222 [00:00<?, ?it/s]

Epoch 2 loss: 249.557775080204


  0%|          | 0/222 [00:00<?, ?it/s]

Epoch 3 loss: 216.76101464033127


  0%|          | 0/222 [00:00<?, ?it/s]

In [ ]:
accuracy = test_model(test_dl, model_tl)
print(f'Test Accuracy: {accuracy*100}%')

In [ ]:
torch.save(model_tl, f='ResNet_Transfer_model.pkl')